# 製品価格予測

（講師は A100 を使用しているが T4 でも実行可能）

## 第7週 三日目：トレーニング！

# 重要なメモ これを読んでください!

以下の pip インストールを実行すると、FSSPECの互換性のないバージョンについて不平を言うPIPからエラーが発生する可能性があります。そのエラーを無視する必要があります！ FSSPECのバージョンは、Hugging Faceが必要とする適切なバージョンです。

ChatGptに尋ねると、FSSPECのより最近のバージョンをインストールすることを推奨してきますが、しかし、それは問題があり、Hugging Faceは、ファイルシステムに関するあいまいなエラーでデータセットを後でロードできないので、pip インストールは以下に表示されているように実行してください。エラーが発生した場合は、逆の方を見てください。

In [1]:
# pip install

!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q --upgrade requests==2.32.3 bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0 datasets==3.2.0 peft==0.14.0 trl==0.14.0 matplotlib wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# import
# Islam S.に感謝します。

import os
import re
import math
from datetime import datetime
from tqdm import tqdm

from google.colab import userdata
from huggingface_hub import login
from datasets import load_dataset, Dataset, DatasetDict

import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed, BitsAndBytesConfig
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig

import wandb
import matplotlib.pyplot as plt

In [3]:
# 定数

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
PROJECT_NAME = "lite-pricer" # "pricer"
HF_USER = "nishi74322014"

# データ
DATASET_NAME = f"{HF_USER}/lite-data"
#DATASET_NAME = "ed-donner/pricer-data" # 後で件数を少なくできる。
MAX_SEQUENCE_LENGTH = 182

# HF Hubにモデルを保存するために名前
RUN_NAME =  f"{datetime.now():%Y-%m-%d_%H.%M.%S}"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"

# QLoRAのハイパーパラメタ
LORA_R = 32        # r:Rank
LORA_ALPHA = 64    # W′ = W + α⋅BA の α
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]
LORA_DROPOUT = 0.1 # α⋅BA の部分に適用される。
QUANT_4_BIT = True # LORA → QLORA

# トレーニング用のハイパーパラメタ

# 必要に応じてより多くのエポックを行うことができる。
# が、1-2で十分、3以上で過学習になる（ログを見て判断）。
EPOCHS = 1

# A100ボックスで16まで設定可能
BATCH_SIZE = 2

# GPUメモリが足りないときに「見かけ上のバッチサイズを大きくする」（→ 精度が安定）
# 「見かけ上のバッチサイズ」= BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS
GRADIENT_ACCUMULATION_STEPS = 2

# 学習率
LEARNING_RATE = 1e-4

# 学習率のスケジュール・タイプ
LR_SCHEDULER_TYPE = 'cosine' # cosineカーブのように徐々に減少

# 学習率ウォームアップ
WARMUP_RATIO = 0.03 # ステップ数の 3% をウォームアップに使う

# 最適化アルゴリズム [paged_adamw_8bit, paged_adamw_32bit, adafactor]
# paged_adamw_32bit：高精度でA100など、paged_adamw_8bit：精度は下がるT4など、adafactor：メモリ使用量は最少、安定性は劣る
# LoRAやQLoRAなどの極端な軽量化が必要な時の最終手段。
OPTIMIZER = "paged_adamw_8bit" # "paged_adamw_32bit"

# 管理者設定 - SAVE_STEPS はHF HUBへの upload (commit) 頻度。
# より頻繁に保存できるよう、5000 から 2000 に変更しました。
LOGGING_STEPS = 50
SAVE_STEPS = 1000 # 2000でもまだ多いので1000に変更
LOG_TO_WANDB = True

%matplotlib inline

In [4]:
# HF Hubにアップロードするモデル名
HUB_MODEL_NAME

'nishi74322014/lite-pricer-2025-08-08_23.34.01'

# オプティマイザーの詳細

https://huggingface.co/docs/transformers/main/en/perf_train_gpu_one#optimizers

最も一般的なのは、AdamまたはAdamW（重減衰のあるAdam）です。

Adamは、以前の勾配のローリング平均を保存することにより、良好な収束を達成します。ただし、モデルパラメタの数の順序の追加メモリフットプリントを追加します。

### Hugging Face と Weights & Biases にログインします

まだHugging Faceアカウントをお持ちでない場合は、https://huggingface.co にアクセスしてサインアップしてトークンを作成します。

次に、左のキーアイコンをクリックして、このノートブックのシークレットを選択し、トークンとして値を持つ `HF_TOKEN` と呼ばれる新しい秘密を追加します。

https://wandb.ai で Weights & Biases についてこれを繰り返し、`WANDB_API_KEY` と呼ばれる秘密を追加する。

In [6]:
# Hugging Faceにログイン

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [7]:
# Weights & Biasesにログイン
wandb_api_key = userdata.get('WANDB_API_KEY')
os.environ["WANDB_API_KEY"] = wandb_api_key
wandb.login()

# プロジェクトに対して記録するように重みとバイアスを構成
os.environ["WANDB_PROJECT"] = PROJECT_NAME
os.environ["WANDB_LOG_MODEL"] = "checkpoint" if LOG_TO_WANDB else "end"
os.environ["WANDB_WATCH"] = "gradients"

wandb: Currently logged in as: nishi_74322014 (nishi_74322014-oss-consortium) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# データをロード

Hugging Faceにアップロードしたので、取得するのは簡単

In [8]:
dataset = load_dataset(DATASET_NAME)
train = dataset['train']
test = dataset['test']

README.md:   0%|          | 0.00/412 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/9.84M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/780k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
# トレーニングデータセットのポイント
print(len(train))

# nishi74322014/lite-data を使用するので削減不要
# トレーニングデータセットを20,000ポイントに削減
#train = train.select(range(20000))

25000


In [ ]:
# wandb.init # 再実行の場合は別バージョンあり。
if LOG_TO_WANDB:
  wandb.init(project=PROJECT_NAME, name=RUN_NAME)

## 次に、トークナイザとモデルをロード

モデルを「量子化」し、精度を4ビットに減らす。

In [11]:
# 適切な量子化を選択

if QUANT_4_BIT:
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
  )
else:
  quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
  )

In [12]:
# トークナイザとモデルをロード

# トークナイザをロード
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# （ベース）モデルをロード
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)

# （バッチ化などで）長さを揃えるために 「pad_token_id」 が必要
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

print(f"Memory footprint: {base_model.get_memory_footprint() / 1e6:.1f} MB")

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Memory footprint: 5591.5 MB


# DataCollator

は、データの前処理（バッチ化）を行うクラスで、このトレーニングでは、

モデルが商品の説明含め予測するのではなく、価格のみを予測するようにトレーニングすることが重要になる。

「Price is $」までの部分は、モデルが次のトークンを予測するためのコンテキストを提供するためのもので、

- トレーナーに「`Price is $`までの部分を学習する必要はない」ことを伝える必要がある。
- また、トレーナーは、モデルに「`Price is $`の後のトークンを予測するよう」に教える必要がある。

マスクを設定することでこれを行うための複雑な方法があるが、幸運なことに、

Hugging Faceは私たちのためにこれを大事にするための非常にシンプルなヘルパークラスを提供します。

In [13]:
# 「Price is $」の後ろに続くテキストだけを学習対象にするように指示

from trl import DataCollatorForCompletionOnlyLM
response_template = "Price is $"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

# そして今

## トレーニング用の構成を設定

2つのオブジェクトを作成

- LoRAのハイパーパラメタを備えた LoraConfig
- 全体的なトレーニングパラメタを備えた SFTConfig

In [14]:
# Googleドライブをマウントして、トレーニング成果物を保存
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
# まず、LoraConfigを設定

lora_parameters = LoraConfig(
    # LoRAスケーリング係数α
    lora_alpha=LORA_ALPHA,
    # LoRA適用部分のドロップアウト率。
    lora_dropout=LORA_DROPOUT,
    # LoRA（の低ランク行列）のランク（次元）。
    r=LORA_R,
    # LoRAでバイアス項をどのように扱うか。
    bias="none",
    # タスク種類：因果言語モデル（次単語予測）
    task_type="CAUSAL_LM",
    # LoRAを適用するモジュール
    target_modules=TARGET_MODULES,
)

# 次に、SFTConfigを設定

train_parameters = SFTConfig(
    # トレーニング成果物保存ディレクトリ名
    # チェックポイントからの再開可能にする。
    output_dir="/content/drive/MyDrive/SFTOutputs/" + PROJECT_NAME,
    # 学習エポック数
    num_train_epochs=EPOCHS,
    # 学習時のバッチサイズ（デバイスごと）
    per_device_train_batch_size=BATCH_SIZE,
    # 評価時のバッチサイズ（デバイスごと）。
    per_device_eval_batch_size=1,
    # 評価を行わない設定。通常 "steps" や "epoch" を指定可能。
    eval_strategy="no",
    # 見かけ上のバッチサイズを大きくする
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    # 使用する最適化アルゴリズム
    optim=OPTIMIZER,
    # モデルを何ステップごとに保存するか？
    save_steps=SAVE_STEPS,
    # 保存するチェックポイントの最大数
    save_total_limit=10,
    # ログを出力するステップ間隔
    logging_steps=LOGGING_STEPS,
    # 学習率
    learning_rate=LEARNING_RATE,
    # 重み減衰（L2正則化）
    weight_decay=0.001,
    # 16ビット浮動小数点（半精度）演算を使わない。
    fp16=False,
    # bfloat16での学習を有効化（T4は非対応）
    bf16=False, # True,
    # 勾配クリッピングの最大値（勾配爆発の防止）
    max_grad_norm=0.3,
    # 総ステップ数をエポック数で制御
    max_steps=-1,
    # ウォームアップステップの割合
    warmup_ratio=WARMUP_RATIO,
    # バッチを同じ長さの入力にまとめ効率化
    group_by_length=True,
    # 学習率スケジューラーのタイプ（説明済み
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    # Weights & Biases（W&B）へのログ送信の有効/無効
    report_to="wandb" if LOG_TO_WANDB else None,
    # W&B上などで使われるRUN_NAME
    run_name=RUN_NAME,
    # モデルが処理する最大トークン長
    max_seq_length=MAX_SEQUENCE_LENGTH,
    # 入力データのテキストが格納されているフィールド名
    dataset_text_field="text",
    # 保存タイミングの基準をステップ数に
    save_strategy="steps",
    # モデルを保存するたびにHubアップロード
    hub_strategy="every_save",
    # 学習済みモデルをHugging Face Hubにアップロード
    push_to_hub=True,
    # Hugging Face Hubにアップロードする際のモデル名
    hub_model_id=HUB_MODEL_NAME,
    # H非公開リポジトリとしてアップロード
    hub_private_repo=True
)

In [ ]:
# 再実行の場合は別バージョンあり。

# そして今、監督されたファインチューニング・トレーナーがファインチューニングを実行します。
# これらの2セットの構成パラメタ（LoraConfig、SFTConfig）を与える。
# TRLの最新バージョンは、ラベルに関する警告を示している - この警告を無視してください
# 良いトレーニング結果が見られない場合はお知らせください（損失が下がっています）。

fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=train,
    peft_config=lora_parameters,
    args=train_parameters,
    data_collator=collator
  )

## 次のセルでは、ファインチューニングを開始します！

これはしばらくの間実行され、Save_Stepsステップで HF HUB にアップロードします。

しばらくすると、GoogleはあなたのColabを止めるかもしれません。

無料プランを使用している人にとって、Googleがリソースが少ないときはいつでも発生する可能性があります。

有料プランの場合は誰でも、最大24時間を与えることができますが、保証はありません。

サーバーが停止した場合は、ここで私のColabをたどって最後の保存から再開できる。

https://colab.research.google.com/drive/1qGTDVIas_Vwoby4UVi2vwsU0tHXy8OMO#scrollTo=R_O04fKxMMT-

このColabを出力での最終実行で保存したので、例を見ることができます。

トリックは、fine_tunedモデルをロードするときに「is_trainable」を設定する必要があることです。

### とにかく、それを念頭に置いて、これをキックオフしましょう！

In [ ]:
# 再実行の場合は別バージョンあり。

# ファインチューニング！
# この設定でGPUメモリ 10GB 程度を消費
fine_tuning.train()

# 結果をHF HUBにプッシュ
fine_tuning.model.push_to_hub(PROJECT_RUN_NAME, private=True)
print(f"Saved to the hub: {PROJECT_RUN_NAME}")

In [ ]:
# W&Bに終了を通知
if LOG_TO_WANDB:
  wandb.finish()

# ファインチューニングの再開
以下の方法でファインチューニングを再開できる。
- HuggingFace Hubに保存されたパラメタを使って
- output_dirに保存されたチェックポイントを使って

以下のように設定する

- pip import 定数 を再実行
- 定数 については、以下に★追加あり（定数は必要に応じ修正）
- Hugging Face と Weights & Biases にログイン（再実行）
- Hugging Faceにアップロードしたデータをロード（再実行）
- wandb.initについては下記の★"別バージョン"を実行
- トークナイザとモデルをロード（意味の無い繰り返しは無視）（再実行）
- DataCollator、LoraConfig、SFTConfigを再構築（再実行）
- SFTTrainer構築については下記の★"別バージョン"を実行
- 最後に、ファインチューニングを再開
  - HuggingFace Hubのパラメタを使って再実行
  - output_dirのチェックポイントを使って：下記の★"別バージョン"を実行


※ 再実行とあるモノは前述のセルから再実行する。

In [ ]:
# pip import 定数 を実行

In [5]:
# 追加の定数 # ★ 再々実行の際に更新忘れがち
# 忘れてもある程度よしなに（最新のコミットから継続実行）してくれる

# モデル名
ORIGINAL_FINETUNED = f"{HF_USER}/{PROJECT_NAME}-2025-08-08_04.45.29"

# リビジョン（branch, tag, commit_id のいずれか）
ORIGINAL_REVISION = "978463dc4b8858660dd01b489f72c792c2352998" # the commit_id in the HuggingFace repo

In [10]:
# wandb.initの別バージョン
if LOG_TO_WANDB:
  details = {
      "base_model": BASE_MODEL,
      "dataset": DATASET_NAME,
      "size": len(train),
      "epochs": EPOCHS,
      "alpha": LORA_ALPHA,
      "r": LORA_R,
      "dropout": LORA_DROPOUT,
      "batch_size": BATCH_SIZE,
      "gradient_accumulation": GRADIENT_ACCUMULATION_STEPS,
      "lr": LEARNING_RATE,
      "lr_scheduler": LR_SCHEDULER_TYPE,
      "max_sequence_length": MAX_SEQUENCE_LENGTH,
      "quant_4_bit": QUANT_4_BIT
  }

  wandb.init(
      project=PROJECT_NAME,
      name=RUN_NAME,
      config=details
  )

In [16]:
# ファインチューニングされたPEFTモデルをロード
if ORIGINAL_REVISION:
  fine_tuned_model = PeftModel.from_pretrained(base_model, ORIGINAL_FINETUNED, revision=ORIGINAL_REVISION, is_trainable=True)
else:
  fine_tuned_model = PeftModel.from_pretrained(base_model, ORIGINAL_FINETUNED, is_trainable=True)

# コレは（、SFTTrainerではないため）、モデルを訓練モードに設定しているだけで、トレーニングは再開されない。
fine_tuned_model.train()

print(f"Memory footprint: {fine_tuned_model.get_memory_footprint() / 1e6:.1f} MB")

adapter_config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

Memory footprint: 5700.6 MB


In [17]:
# 1つ前のセルで fine_tuned_model をロードしておく。
# 監督されたファインチューニング・トレーナーの再構築
fine_tuning = SFTTrainer(
    model=fine_tuned_model, # base_modelでない
    train_dataset=train,
    peft_config=lora_parameters,
    args=train_parameters,
    data_collator=collator
  )

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [18]:
# ステップ・エポック数、オプティマイザの状態を継続させる場合
# ★ SFTConfig(output_dir=PROJECT_NAME...)に結果が保存されている必要がある。
# output_dirは、セッションが切れる（切断→再接続）と消えると言われている。
fine_tuning.train(resume_from_checkpoint=True)

# 結果をHF HUBにプッシュ
fine_tuning.model.push_to_hub(PROJECT_RUN_NAME, private=True)
print(f"Saved to the hub: {PROJECT_RUN_NAME}")

/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:3441: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

Step,Training Loss
5050,1.256300
5100,1.175100
5150,1.229500
5200,1.191700
5250,1.222800
5300,1.268900
5350,1.200000
5400,1.235400
5450,1.236300
5500,1.208500


wandb: Adding directory to artifact (/content/drive/MyDrive/SFTOutputs/lite-pricer/checkpoint-6000)... Done. 1.0s
wandb: Adding directory to artifact (/content/drive/MyDrive/SFTOutputs/lite-pricer/checkpoint-6250)... Done. 4.3s


README.md:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...p0b4zsz9w/adapter_model.safetensors:  38%|###8      | 41.9MB /  109MB            

No files have been modified since last commit. Skipping to prevent empty commit.


Saved to the hub: lite-pricer-2025-08-08_23.34.01
